In [469]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

In [470]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


In [471]:
#데이터셋
DATASET_NAME = 'kfood'
DRIVE_PATH = Path(os.getcwd())
DATASET_PATH = DRIVE_PATH / DATASET_NAME
filepath = DATASET_PATH
print(filepath.exists())

True


In [472]:
#class_label 매칭 딕셔너리로 저장
labels = None
classes = None
with open(filepath / 'class_label.csv','r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()
#print(len(classes), len(labels), classes, labels)
class_to_label = {}
for _class, _label in zip(classes, labels):
    class_to_label[_class] = int(_label)
tf_class_to_label = tf.constant(list(class_to_label.keys()))

In [473]:
def get_image_crop_points(filepath):
    crops = {}
    properties = filepath / "crop_area.properties"
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=")
            if name != "" and crop != "":
                #name = name.encode('utf-8')
                crop = crop.split(",")
                if len(crop) >= 4:
                    crop = [int(crop[1]), int(crop[0]), int(crop[3]), int(crop[2])]
                    crops[name] = crop
                elif len(crop) == 2:
                    crop = [0, 0, int(crop[1]), int(crop[0])]
                    crops[name] = crop
            
    return crops

In [474]:
#crop 지점 정보 빼오기
crop_points = {}
class_list = list(filepath.glob("*/*"))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

tf_crop_image_names = tf.constant(list(crop_points.keys()), dtype=tf.string)
tf_crop_points = tf.constant(list(crop_points.values()))


In [475]:
#데이터셋의 이미지 경로 및 레이블 저장
from glob import glob
image_paths = sorted(glob("kfood/*/*/*"))
image_paths = [image_path for image_path in image_paths if image_path.split("/")[-1].split(".")[-1].lower() in ("png", "jpg", "jpeg")]
#labels = [class_to_label[Path(image_path).parent.stem] for image_path in image_paths]
print(len(image_paths))#, len(labels))


150507


In [476]:
#image_paths = sorted(glob("kfood/*/*/*"))
#image_formats = np.unique(np.array([image_path.split("/")[-1].split(".")[-1].lower() for image_path in image_paths]))
#print(image_formats)

In [477]:
def parse_and_crop_image_add_label(tf_filepath):
    
    image = tf.io.read_file(tf_filepath) # 이미지 파일 읽기
    filepath = tf.compat.path_to_str(tf_filepath)
    #format decoding
    image_format = tf.strings.lower(tf.strings.split(filepath, ".")[-1])

    if image_format == "jpeg":
        image = tf.image.decode_jpeg(image, channels=3) # JPEG-encoded -> uint8 tensor (RGB format)
    elif image_format == "png":
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)
    else:    
        image = tf.image.decode_image(image, channels=3, expand_animations=False)


    #crop
    image_name = tf.strings.split(tf.strings.split(filepath, "/")[-1], ".")[0]
    tf_image_idx = tf.where(tf_crop_image_names == image_name)
    
    #crop 정보가 있으면 크롭
    if tf.reduce_all(tf.not_equal(tf.shape(tf_image_idx), tf.constant((0, 1), dtype=tf.int32))):
        crop_offsets = tf_crop_points[tf.reshape(tf_image_idx, shape=())]
        image = tf.image.crop_to_bounding_box(image, crop_offsets[0], crop_offsets[1], crop_offsets[2], crop_offsets[3])
    
    #labeling
    class_name = tf.strings.split(filepath, "/")[-2]
    tf_class_name_idx = tf.where(tf_class_to_label == class_name)
    try:
        label = tf.reshape(tf_class_name_idx, shape=())
    except:
        label = 0
        print("label error")

    return image, int(label)

In [478]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 2
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 2
    right_crop = shape[1] - left_crop
    return image[top_crop : bottom_crop, left_crop : right_crop]


def resizing_image(image, label):
    image = central_crop(image)
    image = tf.cast(image, tf.uint8)
    return tf.image.resize(image, [224, 224], method="nearest"), label

In [479]:
def make_kfood_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None, n_parse_threads=5, batch_size=32, cache=False):

    filenames_dataset = tf.data.Dataset.from_tensor_slices(filepaths)
    dataset = filenames_dataset.map(parse_and_crop_image_add_label, num_parallel_calls=n_parse_threads)
    dataset = dataset.map(resizing_image, num_parallel_calls=n_parse_threads)
    #dataset = filenames_dataset.map(spa)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [480]:
#데이터셋 생성
dataset = make_kfood_dataset(image_paths, shuffle_buffer_size=10000, n_parse_threads=tf.data.AUTOTUNE)

In [481]:
for images, labels in dataset.take(1):
    plt.figure(figsize=(16,8))
    plt.axis("off")
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.imshow(images[i]/255.)
        print(bytes(tf_class_to_label[int(labels[i])].numpy()).decode('utf-8'))
            

[[[192 213 218]
  [192 213 218]
  [192 213 218]
  ...
  [209 229 236]
  [209 229 236]
  [209 229 236]]

 [[192 213 218]
  [192 213 218]
  [192 213 218]
  ...
  [209 229 236]
  [209 229 236]
  [209 229 236]]

 [[192 213 218]
  [192 213 218]
  [192 213 218]
  ...
  [210 230 237]
  [210 230 237]
  [210 230 237]]

 ...

 [[133 149 149]
  [132 148 148]
  [131 147 147]
  ...
  [213 232 239]
  [213 232 239]
  [213 232 239]]

 [[134 150 150]
  [133 149 149]
  [132 148 148]
  ...
  [213 231 241]
  [213 231 241]
  [213 231 241]]

 [[135 151 151]
  [134 150 150]
  [133 149 149]
  ...
  [213 231 241]
  [213 231 241]
  [213 231 241]]]
[[[232 233 227]
  [232 233 227]
  [232 233 227]
  ...
  [230 231 225]
  [230 231 225]
  [230 231 225]]

 [[232 233 227]
  [232 233 227]
  [232 233 227]
  ...
  [230 231 225]
  [230 231 225]
  [230 231 225]]

 [[232 233 227]
  [232 233 227]
  [232 233 227]
  ...
  [230 231 225]
  [230 231 225]
  [230 231 225]]

 ...

 [[195 188 180]
  [194 187 179]
  [186 179 171]
  ..

2021-12-31 14:58:54.337643: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


[[[179 136 156]
  [178 135 155]
  [175 132 151]
  ...
  [146 143 128]
  [145 141 129]
  [144 140 128]]

 [[176 133 153]
  [173 130 149]
  [168 125 144]
  ...
  [146 143 128]
  [145 141 129]
  [144 140 128]]

 [[176 131 151]
  [169 124 144]
  [162 117 137]
  ...
  [147 144 129]
  [145 141 129]
  [144 140 128]]

 ...

 [[170 165 143]
  [170 165 143]
  [169 164 142]
  ...
  [149 141 118]
  [150 142 121]
  [150 142 121]]

 [[172 167 145]
  [172 167 145]
  [173 168 146]
  ...
  [148 140 119]
  [144 136 117]
  [144 136 117]]

 [[174 169 147]
  [174 169 147]
  [175 170 148]
  ...
  [150 144 122]
  [149 141 122]
  [149 141 122]]]
[[[185 197 185]
  [185 197 185]
  [189 199 188]
  ...
  [178 185 177]
  [180 185 178]
  [183 188 181]]

 [[182 194 182]
  [181 193 181]
  [186 196 185]
  ...
  [170 177 169]
  [174 179 172]
  [178 183 176]]

 [[180 192 180]
  [179 191 179]
  [183 193 182]
  ...
  [178 185 177]
  [179 186 178]
  [181 188 180]]

 ...

 [[237 247 236]
  [235 245 234]
  [226 236 225]
  ..

Error in sys.excepthook:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:



Original exception was:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:


Error in sys.excepthook:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:



Original exception was:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:




[[[227 228 230]
  [218 219 221]
  [218 218 220]
  ...
  [199 206 212]
  [201 206 210]
  [203 208 212]]

 [[232 233 235]
  [226 227 229]
  [225 225 227]
  ...
  [205 210 214]
  [203 208 212]
  [204 209 213]]

 [[232 233 235]
  [223 224 226]
  [217 217 219]
  ...
  [211 214 219]
  [207 210 215]
  [206 210 213]]

 ...

 [[138 86 47]
  [118 65 33]
  [110 57 26]
  ...
  [216 217 219]
  [221 222 224]
  [221 222 224]]

 [[143 86 56]
  [114 56 34]
  [117 58 40]
  ...
  [222 223 225]
  [224 225 229]
  [220 221 225]]

 [[189 129 103]
  [148 87 69]
  [141 82 68]
  ...
  [226 227 229]
  [228 229 233]
  [223 224 228]]]
[[[82 58 54]
  [81 57 53]
  [82 57 53]
  ...
  [162 159 230]
  [161 160 228]
  [161 160 228]]

 [[77 56 55]
  [75 54 53]
  [73 52 51]
  ...
  [164 158 230]
  [163 160 229]
  [163 160 229]]

 [[67 48 50]
  [64 45 47]
  [61 42 44]
  ...
  [163 157 229]
  [165 159 231]
  [166 160 232]]

 ...

 [[180 189 206]
  [180 189 206]
  [179 188 205]
  ...
  [202 199 226]
  [204 197 230]
  [204 19

...

[[[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [13 17 16]
  [13 17 16]
  [13 17 16]]

 [[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [14 18 17]
  [14 18 17]
  [14 18 17]]

 [[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [14 18 17]
  [14 18 17]
  [14 18 17]]

 ...

 [[33 32 37]
  [34 33 38]
  [34 33 38]
  ...
  [31 29 32]
  [31 29 32]
  [31 29 32]]

 [[34 33 38]
  [36 35 40]
  [36 35 40]
  ...
  [31 31 33]
  [30 30 32]
  [30 30 32]]

 [[35 34 39]
  [37 36 41]
  [38 37 42]
  ...
  [31 32 34]
  [31 32 34]
  [30 31 33]]]
[[[107 168 127]
  [106 167 126]
  [107 168 127]
  ...
  [70 47 39]
  [72 50 39]
  [75 53 42]]

 [[111 169 129]
  [110 168 128]
  [110 168 128]
  ...
  [69 46 38]
  [70 47 39]
  [72 50 39]]

 [[116 172 133]
  [114 170 131]
  [113 169 130]
  ...
  [61 41 34]
  [62 42 35]
  [64 44 35]]

 ...

 [[92 62 38]
  [87 57 31]
  [82 52 26]
  ...
  [15 15 13]
  [14 14 14]
  [15 15 15]]

 [[90 63 36]
  [101 74 47]
  [110 80 54]
  ...
  [14 14 14]
  [14 14 16]
  [14 14 16]]

 [[118 92 65]
  [103 77 50]
  [113 83 55

Error in sys.excepthook:
Error in sys.excepthook:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:



Original exception was:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:




176]
  ...
  [221 221 223]
  [232 232 234]
  [16 16 18]]

 [[21 20 15]
  [182 181 176]
  [195 194 189]
  ...
  [230 230 232]
  [225 225 227]
  [21 21 23]]

 [[4 3 0]
  [18 17 12]
  [14 13 8]
  ...
  [9 9 11]
  [22 22 24]
  [4 4 6]]]
[[[204 209 212]
  [201 206 209]
  [201 206 209]
  ...
  [216 224 226]
  [216 224 226]
  [215 223 225]]

 [[177 182 185]
  [185 190 193]
  [195 200 203]
  ...
  [216 224 226]
  [217 225 227]
  [217 225 227]]

 [[151 156 159]
  [168 173 176]
  [188 193 196]
  ...
  [215 223 225]
  [217 225 227]
  [218 226 228]]

 ...

 [[205 212 218]
  [200 208 211]
  [203 211 214]
  ...
  [211 220 227]
  [219 228 235]
  [205 214 221]]

 [[204 213 220]
  [198 207 212]
  [202 211 216]
  ...
  [209 220 226]
  [213 224 230]
  [219 228 235]]

 [[200 209 216]
  [203 212 219]
  [207 216 221]
  ...
  [213 224 230]
  [212 223 229]
  [210 219 226]]]
[[[204 209 212]
  [201 206 209]
  [201 206 209]
  ...
  [216 224 226]
  [216 224 226]
  [215 223 225]]

 [[177 182 185]
  [185 190 193]
 


Original exception was:
Error in sys.excepthook:

Original exception was:
Error in sys.excepthook:

Original exception was:
Error in sys.excepthook:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:



Original exception was:
Traceback (most recent call last):


During handling of the above exception, another exception occurred:




[[[209 207 194]
  [209 207 194]
  [209 207 194]
  ...
  [252 252 252]
  [253 253 253]
  [254 254 254]]

 [[210 208 195]
  [209 207 194]
  [208 206 193]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[208 209 195]
  [207 208 194]
  [204 205 191]
  ...
  [249 249 249]
  [250 250 250]
  [252 252 252]]

 ...

 [[247 250 255]
  [247 250 255]
  [249 250 254]
  ...
  [232 233 235]
  [232 233 235]
  [232 233 235]]

 [[248 252 255]
  [248 252 255]
  [248 252 255]
  ...
  [236 237 239]
  [236 237 239]
  [236 237 239]]

 [[247 251 254]
  [247 251 254]
  [247 251 254]
  ...
  [235 236 238]
  [235 236 238]
  [236 237 239]]]


In [ ]:
image = tf.image.resize(images[0], (255, 555))
print(tf.shape(image))
print(tf.shape(tf.image.central_crop(image, 0.9)))
print(tf.shape(image[0:224, 0:224]))

tf.Tensor([255 555   3], shape=(3,), dtype=int32)
tf.Tensor([231 501   3], shape=(3,), dtype=int32)
tf.Tensor([224 224   3], shape=(3,), dtype=int32)


In [ ]:
image = images[0]

tf_image_idx = tf.where(tf_crop_image_names == "image_name")
#print(tf.equal(tf_image_idx))
tf.reduce_all(tf.equal(tf_image_idx.shape, (0, 1)))

#tf.constant([], shape=(0, 1), dtype=tf.int64)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
image = tf.image.resize(image, [250, 224])
print(image.shape)
image = central_crop(image)
print(image.shape)

(250, 224, 3)
(224, 224, 3)
